### Document Preprocessing: Converting PDFs to TSV file
The papers are stored as PDF files. This notebook combines and converts them to a TSV format that is easier for Snorkel to process.

In [1]:
import numpy as np
import pandas as pd

In [2]:
# Apache Tika package to extract text from PDFs
import tika
from tika import parser

In [3]:
from pathlib import Path

pathlist = Path(r'more_pdfs_epicenter').glob('**/*.pdf')    # pdfs stored in this folder

dict_list = []
paths_list = []

# for loop to parse each documents' contents
for i, path in enumerate(pathlist):
    path_in_str = str(path)
    paths_list.append(path)
    parsed = parser.from_file(path_in_str)
    parsed["doc_name"] = "doc_" + str(i)
    dict_list.append(parsed)

In [4]:
# make csv of paths pointing to PDF file locations
paths_df = pd.DataFrame(paths_list)
paths_df.to_csv('paths.csv', header = False, index = False)

In [5]:
# number of docs
print(len(dict_list))

88


In [6]:
# write the file names and documents' contents to a dataframe
file_names = []
contents = []

for doc in dict_list:
    file_names.append(doc['metadata']['resourceName'])
    contents.append(doc['content'])
    
df = pd.DataFrame({'file_name':file_names, 'content': contents})

In [7]:
# remove newline characs
df.replace(r'\n', ' ', regex = True, inplace = True)
# strip leading whitespace
df['content'] = df['content'].str.lstrip()
# remove other whitespace
df.replace(r'\s\s+', ' ', regex = True, inplace = True)
df.replace(r'\t', ' ', regex = True, inplace = True)

In [8]:
# example of the first documents' contents
print(df.iloc[0]['content'])

ISOLATION OF VIRUSES FROM WILD MAMMALS IN WEST AFRICA, 1966-1970 Journal of Wildlife Diseases Vol. 10, July, 1974 27’? ISOLATION OF VIRUSES FROM WILD MAMMALS IN WEST AFRICA, 1966-1970 GRAHAM E. KEMP*, OTTIS R. CAUSEY*, HENRY W. SETZERt, and DOROTHY 1. MOORE* Abstract: During the S-year period 1966-1970, a total of 7497 wild mammals of at least 101 different species were collected from 36 locations in Nigeria, Dahomey, and Togo and sampled for virus. The collections were made in five ecologically distinct vegetative zones: high forest, Guinea, Sudan, and Sahel woodland, and the J05 Plateau. Sixteen different virus types, represented by 83 isolates, were recovered, as follows: Arumowot (6 isolates), Bhanja (2), bluetongue type 7 (1), Chandipura (1), Congo (2), Dakar bat (3), Dugbe (1), IbAn 17143 (1), IbAn 33709 (1), Lebombo (1), Mokola (4), poxvirus IbAn 34325 (1), Semliki Forest (1), SudAn 754/6 1 (53), Uganda 5 (3), and West Nile (2). Viruses were isolated from Nigeria, the principal 

In [9]:
# export to tsv file
df.to_csv('pdfs_big.tsv', sep='\t', header = False, index = False, encoding = 'utf-8')

## NOTE: After exporting, the format may not save perfectly, we will edit out extra rows in Excel, then save the tsv file, before reading into Snorkel notebook #1

In [25]:
# saving metadata
metadata = pd.read_csv(r'Flaviviruses_v2.txt', sep = '\t', encoding = "cp1252")

In [26]:
# clean up file column
metadata['file'] = metadata['file'].str[26:]

In [27]:
metadata = metadata.rename(columns = {'file':'file_name'})

In [28]:
metadata['file_name'] = metadata['file_name'].astype(str)
df['file_name'] = df['file_name'].astype(str)

In [29]:
df2 = df.merge(metadata, how = 'left', on = 'file_name')

In [30]:
# final dataframe of file title, file content, and metadata info all in one table
df2

file_name  \
0                              0090-3558-10.3.279.pdf   
1                                    059.041.0109.pdf   
2                                   09-642913p460.pdf   
3                    1-s2.0-0035920362900743-main.pdf   
4                   1-s2.0-S0147957197000246-main.pdf   
5                   1-s2.0-S1090023313005315-main.pdf   
6                   1-s2.0-S1877959X13000587-main.pdf   
7                   1-s2.0-S1877959X13001179-main.pdf   
8                                1040638711433325.pdf   
9                                1040638712452723.pdf   
10                                      107-5-444.pdf   
11                                      107-5-444.pdf   
12                                              2.pdf   
13                                       218.full.pdf   
14           9eadd392caba3df5b5fe5fdb6a7b051a13d4.pdf   
15  Achazi-2011-Rodents as Sentinels for the Preva...   
16  Aguirre-1992-Serologic survey for selected arb...   
17  Aitken-1956-The isolation of Ilheus virus from...   
18  Balseiro-2012-Louping ill in goats, Spain, 201...   
19  Bardos-1983-Prevalence of arbovirus antibodies...   
20  Ben Hassine-2014-First detection of co-circula...   
21          Bosch-2007-West Nile virus, Venezuela.pdf   
22  Briggs-2011-Tick-borne encephalitis virus, Kyr...   
23  Buck-1993-Isolation of St. Louis encephalitis.pdf   
24  Buckley-2003-Serological evidence of West Nile...   
25  Calisher-2006-Bats_ important reservoir hosts.pdf   
26  Calzolari-2013-Usutu virus persistence and Wes...   
27         Cao-2011-Tembusu virus in ducks, china.pdf   
28  Cigarroa-Toledo-2016-Serologic Evidence of Fla...   
29  de Thoisy-2009-Dengue infection in neotropical...   
..                                                ...   
61  Pattnaik-2006-Kyasanur forest disease_ an epid...   
62  Pauvolid-Correa-2014-Serological Evidence of W...   
63  Pauvolid-Correa-2014-Serological Evidence of W...   
64  Prow-2013-Natural exposure of horses to mosqui...   
65  Quaglia-2014-West Nile and st. Louis encephali...   
66  Reisen-2016-Surveys for antibodies against mos...   
67  Rizzoli-2007-West Nile virus circulation detec...   
68  Rodrigues-2010-Molecular epidemiology of Saint...   
69  Root-2013-West Nile virus associations in wild...   
70  Sabattini-1965-Serological survey for arboviru...   
71  Scherret-2001-The relationships between West N...   
72        Serological_evidence_of_continuing_high.pdf   
73  Seymour-1983-Serologic evidence of natural tog...   
74  Sotomayor-Bonil-2014-Dengue virus in bats from...   
75  Steinmetz-2011-Emergence and establishment of.pdf   
76  Suss-2007-Tickborne encephalitis in naturally.pdf   
77  Süss-2011-Tick-borne encephalitis 2010_ epidem...   
78  Takeda-1999-Isolation of tick-borne encephalit...   
79  Tavares-Neto-2004-Serologic survey for yellow.pdf   
80  Tavares-Neto-2004-[Serologic survey for yellow...   
81  Timoney-1976-Encephalitis caused by louping il...   
82                                   TM0050040621.pdf   
83  Ubico-1995-Serologic survey of neotropical bat...   
84  Varelas-Wesley-1982-Antigenic relationships of...   
85                                  vbz.2014.1746.pdf   
86                                  vbz.2014.1746.pdf   
87  Wang-2009-Japanese encephalitis viruses from b...   
88      Weissenbock-2013-Usutu virus, Italy, 1996.pdf   
89  Wolfe-2001-Sylvatic transmission of arboviruse...   
90  Yan-2011-An infectious disease of ducks caused...   

                                              content  \
0   ISOLATION OF VIRUSES FROM WILD MAMMALS IN WEST...   
1   BioOne Complete (complete.BioOne.org) is a ful...   
2   SoutheaSt aSian J trop Med public health 460 V...   
3   PII: 0035-9203(62)90074-3 504 TRANSACTIONS OF ...   
4   PII: S0147957197000246 Epidemiology of tick-bo...   
5   Detection of Usutu virus in a bullfinch (Pyrrh...   
6   Detection and genetic characterization of tick...   
7   Surveillance of tick-borne encephalitis virus ...

In [31]:
# export metadata
df2.to_csv('metadata_big.tsv', sep='\t', header = True, index = False, encoding = 'utf-8')

## Extracting more info from the text (eg: virus abbreviations)

In [32]:
import re
#example = "hello 2131242 ffgg (TBEV) f fdr (ABCB) fsdr"
#match = re.findall("\([a-zA-Z]{2,6}?\)", example)
#print(match)

#example2 = "(USA) USA ERF Fegrd"
#match2 = re.findall("\(?USA\)?", example2)
#print(match2)

In [33]:
matches = df2['content'].str.findall("\([A-Z]{3,4}?\)").tolist()
virus_abbrev = []
def removeNestings(l): 
    for i in l: 
        if type(i) == list: 
            removeNestings(i) 
        else: 
            virus_abbrev.append(i) 
  
removeNestings(matches)
virus_abbrev = list(set(virus_abbrev))

In [34]:
virus_abbrev_series = pd.Series(virus_abbrev)

In [35]:
# remove parentheses around the virus acronym
no_paren = virus_abbrev_series.str.replace("\(|\)", "")

In [36]:
# replace some virus acronym (ending with letter V) with a fuller virus acronym
# eg,JEV => JE virus
just_v = no_paren[no_paren.str.endswith('V')].str.replace(r".$", " virus")

In [37]:
# combine all acronyms to one df
no_paren = no_paren.append(just_v)
virus_abbrev_series = virus_abbrev_series.append(no_paren)

In [38]:
# Remove false acronyms that are not virus names
virus_abbrev_series.drop(virus_abbrev_series[virus_abbrev_series.str.contains('\)?MAY\)?')].index, inplace=True)
virus_abbrev_series.drop(virus_abbrev_series[virus_abbrev_series.str.contains('\)?USA\)?')].index, inplace=True)
virus_abbrev_series.drop(virus_abbrev_series[virus_abbrev_series.str.contains('\)?PRNT\)?')].index, inplace=True)
virus_abbrev_series.drop(virus_abbrev_series[virus_abbrev_series.str.contains('\)?UVRI\)?')].index, inplace=True)
virus_abbrev_series.drop(virus_abbrev_series[virus_abbrev_series.str.contains('\)?con\)?')].index, inplace=True)
virus_abbrev_series.drop(virus_abbrev_series[virus_abbrev_series.str.contains('\)?PCR\)?')].index, inplace=True)
virus_abbrev_series.drop(virus_abbrev_series[virus_abbrev_series.str.contains('\)?CAL\)?')].index, inplace=True)
virus_abbrev_series.drop(virus_abbrev_series[virus_abbrev_series.str.contains('\)?EHD\)?')].index, inplace=True)
virus_abbrev_series.drop(virus_abbrev_series[virus_abbrev_series.str.contains('\)?PBS\)?')].index, inplace=True)


In [39]:
# export virus abbreviations to csv file 
virus_abbrev_series.to_csv('virus_abbrev.csv', sep = ",", index = False, header = False)